<a href="https://colab.research.google.com/github/AhmedSSoliman/Leveraging-Pretrained-Language-Models-for-Code-Generation/blob/master/DistilBERT_Marian_CoNaLa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [ ]:
%%capture
!pip install tensorflow
!pip install keras
!pip install netron
!pip install nltk

!pip install git+https://github.com/huggingface/transformers
!pip install datasets
!pip install nlp
!pip install rouge-score
#!pip install sacrebleu
!pip install git-python
!pip install sentencepiece
#!pip install torchdata
#!pip install transformers
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
%cd /content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian


# **Load Data**

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/CoNaLa-Large")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/AhmedSSoliman___csv/AhmedSSoliman--CoNaLa-Large-87e6f6b36beed736/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 24687
    })
    test: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 1237
    })
})

# **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast = True)
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# **Model**

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("distilbert-base-uncased", "Helsinki-NLP/opus-mt-en-nl")
model

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-nl were not used when initializing MarianForCausalLM: ['model.encoder.layers.5.fc1.weight', 'model.encoder.layers.1.self_attn.out_proj.weight', 'model.encoder.layers.2.self_attn.k_proj.bias', 'model.encoder.layers.5.self_attn.v_proj.bias', 'model.encoder.layers.0.self_attn.q_proj.bias', 'final_logits_bias', 'model.encoder.layers.4.self_attn.out_proj.weight', 'model.encoder.layers.5.final_layer_norm.bias', 'model.encoder.embed_positions.weight', 'model.encoder.layers.1.self_attn_layer_norm.weight', 'model.encoder.layers.1.self_attn.v_proj.weight', 'model.encoder.layers.5.self_attn.out_proj.bias', 'model.encoder.layers.5.self_attn.out_proj.weight', 'model.encoder.layers.1.fc2.bias', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.4.self_attn.k_proj.weight', 'model.encoder.layers.0.final_layer_norm.bias', 'model.encoder.layers.2.self_attn.q_proj.weight', 'model.encoder.layers.2.self_attn.v_proj.bias', 

EncoderDecoderModel(
  (encoder): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 8,
    "encoder_ffn_dim": 2048,
    "encoder_layerdrop": 0.

In [ ]:
model.base_model.config.encoder.num_hidden_layers = 6

In [ ]:
model.tie_weights = True

In [ ]:
model.base_model.config.decoder_start_token_id = model.base_model.config.decoder.bos_token_id

In [ ]:
model.base_model.config.bos_token_id = tokenizer.bos_token_id
model.base_model.config.eos_token_id = tokenizer.eos_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model.config.max_length =512

In [ ]:
model.save_pretrained("distilBERT-Marian-base-model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("./distilBERT-Marian-base-model")
#model=AutoModelForSeq2SeqLM.from_pretrained("./distilbert-marian-training")
model

EncoderDecoderModel(
  (encoder): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear

In [ ]:
model.base_model.config.decoder_start_token_id

0

In [ ]:
model.base_model.config.pad_token_id

0

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
model.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "./distilBERT-Marian-base-model",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_sto

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian




# **Prepare the data.**

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["snippet"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



  0%|          | 0/24687 [00:00<?, ?ba/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


  0%|          | 0/1237 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./distilbert-marian-training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

Using cuda_amp half precision backend


In [ ]:
# distilbert-marian-training
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24687
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 123440
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2280: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarnin

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.664900,1.604529,7.364900,6.694330,25.942304,10.589395,4.541960,7.541722,16.949651,23.487875,9.258663,4.189167
2,1.277900,1.251232,16.632170,15.532539,45.845406,22.374882,11.741704,11.518789,24.557738,43.230831,19.787177,11.123135
3,1.093400,1.061539,19.616163,18.542513,49.132290,26.096812,14.782982,14.115106,28.441479,46.620697,23.634101,14.232709
4,0.932200,0.944360,25.302316,24.173222,52.010016,30.297183,19.439741,19.354470,33.910725,49.746761,28.071721,18.920266
5,0.779300,0.855455,29.695529,28.765095,55.023066,33.892840,23.366746,22.819473,37.296880,52.942861,31.843547,22.879567
6,0.683600,0.785743,32.884157,31.888821,58.265192,38.334894,27.469816,26.766172,41.777872,56.319500,36.404506,27.125071
7,0.633100,0.736064,36.175505,35.291062,60.827353,41.570404,31.320635,29.705211,44.009661,58.985137,39.800491,31.069736
8,0.536400,0.702253,38.653911,37.768210,61.585086,43.181929,33.158931,32.743573,46.489073,59.825755,41.445243,32.872974
9,0.542000,0.683848,40.630171,39.738212,63.050135,44.889556,35.002834,33.980664,48.087469,61.367288,43.222798,34.667552
10,0.515300,0.674945,41.612979,40.729268,63.740023,45.707657,35.905609,35.382852,49.149353,62.120255,44.088746,35.565501


***** Running Evaluation *****
  Num examples = 1237
  Batch size = 1
INFO:seq_to_seq:Got BLEU of 7.36 and SacreBLEU of 6.69
Saving model checkpoint to ./distilbert-marian-training/checkpoint-12344
Configuration saved in ./distilbert-marian-training/checkpoint-12344/config.json
Model weights saved in ./distilbert-marian-training/checkpoint-12344/pytorch_model.bin
tokenizer config file saved in ./distilbert-marian-training/checkpoint-12344/tokenizer_config.json
Special tokens file saved in ./distilbert-marian-training/checkpoint-12344/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2280: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWar

TrainOutput(global_step=123440, training_loss=1.014285834084373, metrics={'train_runtime': 17647.292, 'train_samples_per_second': 13.989, 'train_steps_per_second': 6.995, 'total_flos': 3230087590379520.0, 'train_loss': 1.014285834084373, 'epoch': 10.0})

In [ ]:
# distilbert-marian-training-1
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24687
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 123440
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2280: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarnin

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.530100,0.700547,39.525685,38.736364,63.261982,44.566237,34.507783,33.357210,47.290011,61.487759,42.869965,34.297334
2,0.436300,0.630652,44.075792,43.281814,65.815616,48.638266,39.453535,37.934901,51.374842,64.202412,47.097216,39.105957
3,0.398300,0.584406,47.640653,46.828668,68.462470,52.275310,43.416482,41.918448,54.326235,67.024555,50.881839,43.238495
4,0.341800,0.552617,51.325929,50.597264,70.368997,55.485616,47.368972,45.802242,57.842923,69.060336,54.306485,47.286865
5,0.283800,0.516560,53.296095,52.605592,71.762267,57.167286,49.035412,48.216962,59.611486,70.512506,56.006069,48.951166
6,0.246100,0.495088,56.198715,55.641908,72.281113,58.651825,51.109419,50.265751,61.216032,71.062992,57.608085,51.078784
7,0.236000,0.476866,57.781199,57.195008,74.233971,61.054826,53.630363,52.188326,63.129419,73.089474,60.065305,53.552639
8,0.201400,0.460722,59.598575,59.059067,75.739427,63.173580,56.195081,54.010902,64.555632,74.651961,62.145802,56.110894
9,0.208700,0.452918,59.228701,58.702182,74.832475,62.028986,55.132552,53.994814,64.489419,73.725566,61.052632,55.059524
10,0.200300,0.448161,60.284862,59.776739,75.381184,62.988328,56.251923,54.858141,65.285968,74.302461,62.075540,56.221452


***** Running Evaluation *****
  Num examples = 1237
  Batch size = 1
INFO:seq_to_seq:Got BLEU of 39.53 and SacreBLEU of 38.74
Saving model checkpoint to ./distilbert-marian-training-1/checkpoint-12344
Configuration saved in ./distilbert-marian-training-1/checkpoint-12344/config.json
Model weights saved in ./distilbert-marian-training-1/checkpoint-12344/pytorch_model.bin
tokenizer config file saved in ./distilbert-marian-training-1/checkpoint-12344/tokenizer_config.json
Special tokens file saved in ./distilbert-marian-training-1/checkpoint-12344/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2280: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:53

TrainOutput(global_step=123440, training_loss=0.31656123802249336, metrics={'train_runtime': 24698.2128, 'train_samples_per_second': 9.995, 'train_steps_per_second': 4.998, 'total_flos': 3230087590379520.0, 'train_loss': 0.31656123802249336, 'epoch': 10.0})

In [ ]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./distilbert-marian-training1
Configuration saved in ./distilbert-marian-training1/config.json
Model weights saved in ./distilbert-marian-training1/pytorch_model.bin
tokenizer config file saved in ./distilbert-marian-training1/tokenizer_config.json
Special tokens file saved in ./distilbert-marian-training1/special_tokens_map.json


# **Visualize**

In [ ]:
%load_ext tensorboard

### **distilBERT-marian**

In [ ]:
tensorboard --logdir ./distilbert-Marian-Training/runs

<IPython.core.display.Javascript object>

# **Inference**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained("./distilbert-marian-training1/")
new_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("./distilbert-marian-training1/", use_fast=True)


In [ ]:
new_model

EncoderDecoderModel(
  (encoder): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear

In [ ]:
new_model.config.max_length


512

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

In [ ]:
batch_size = 1

# map data correctly
def generate_new_code(batch):
    inputs = new_tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch


In [ ]:
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

#results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size, remove_columns=["intent"])

  0%|          | 0/500 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1268: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilBERT-Marian


In [ ]:
import re
import string

import datasets
import numpy as np

import evaluate
from evaluate import load

In [ ]:

exact_match = evaluate.load("exact_match")

#refs = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
#preds = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

"""
for i in range(len(results)):
  refs = results["code"][i]
  preds = results["pred_code"][i]
"""

'\nfor i in range(len(results)):\n  refs = results["code"][i]\n  preds = results["pred_code"][i]\n'

In [ ]:
refs = results["snippet"]
preds = results["pred_code"]

In [ ]:

exact_match_results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["▁", " "], ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
print(round(exact_match_results["exact_match"], 5)*100)

12.4


In [ ]:
with open("./references.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(ref + '\n')
    else:
      continue


with open("./predictions.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(pred + '\n')
    else:
      continue


In [ ]:
# Ok Evaluator
bleu_score =0
rouge_score = 0
sacre_bleu = 0

for i in range(len(results)):
  ref = results["snippet"][i]
  pred = results["pred_code"][i]
  if pred is not None and pred != "":
      if ref is not None and ref != "":
        #bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
        #bleu_score  += bleu_metric["score"]

        bleu_metric = evaluator.evaluate([pred], [ref])
        #bleu_metric = evaluator.evaluateSingle(pred, ref)

        #bleu_score += bleu_metric['BLEU-Unigram-Precision']
        bleu_score += bleu_metric['BLEU']
        rouge_score += bleu_metric['ROUGE-L']
        sacre_bleu += bleu_metric['SacreBLEU']
        #print(bleu_metric['SacreBLEU'])
  else:
    continue

# **Performance Metrics**

**BLEU Score**

In [ ]:
#distilbert-marian
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))


Bleu Score: 32.46176998338745 (scale 0-100)


**SacreBLEU**

In [ ]:
print('Sacre Bleu: {} (scale 0-100)'.format(sacre_bleu/len(results)))

Sacre Bleu: 29.478990662734873 (scale 0-100)


**Rouge Score**

In [ ]:
#distilbert-marian
print('ROUGE Score: {} (scale 0-100)'.format(rouge_score/len(results)))


ROUGE Score: 43.94671098918023 (scale 0-100)


**CodeBLEU**

In [ ]:
from google.colab import files

In [ ]:
import calc_code_bleu
ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score, code_bleu_score = calc_code_bleu.calc_code_bleu("references.txt","predictions.txt","python", "keywords")

In [ ]:
print("ngram_match_score: ", ngram_match_score*100)
print("weighted_ngram_match_score: ", weighted_ngram_match_score*100)
print("syntax_match_score: ", syntax_match_score*100)
print("dataflow_match_score: ", dataflow_match_score*100)
print("code_bleu_score: ", code_bleu_score*100)

# **Save the  results**

In [ ]:
## %cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa-Large


In [ ]:
pwd

'/content/drive/MyDrive/models/CodeGeneration/New/CoNaLa/distilRoBERTa-Marian'

In [ ]:
bleu_score = 0
with open("./Output.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["intent"][i]
    ref = results["snippet"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":

        bleu_metric = evaluator.evaluate([pred], [ref])
        f.write( "NL: " + nl + '\n')
        f.write( "ref: " + ref + '\n')
        f.write( "Output: " + pred + '\n')
        f.write("bleu_metric: " + str(bleu_metric["BLEU"]) + '\n')
        f.write(10*"-" + '\n')
        bleu_score  += bleu_metric["BLEU"]

    else:
      continue


# **Test**

In [ ]:
# map data correctly
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code

In [ ]:

#NL = "decode a hex string to utf-8."
NL = "remove first and last lines of string `s`"
output_code = generate_code(NL)

In [ ]:
output_code

In [ ]:
natural_language = "How I can get rid of None values in dictionary?"
gen_code = generate_code(natural_language)

In [ ]:
gen_code